In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import urllib.request
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import os

In [ ]:
#inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
#urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(weights=local_weights_file,
                                    input_shape=(150, 150, 3),
                                    include_top=False)

for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.output

In [ ]:
train_dir = '/content/drive/MyDrive/datasets/Train' #os.path.join(dataset, "Train")
validation_dir = '/content/drive/MyDrive/datasets/Test' #os.path.join(dataset, "Test")

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=128,
                                                    class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(validation_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='binary')

x = last_layer
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

Found 1420 images belonging to 2 classes.
Found 720 images belonging to 2 classes.


In [ ]:
def solution_A3():
  model.fit(train_generator,
          epochs=20,
          steps_per_epoch=8,
          verbose=1,
          validation_data=val_generator,
          validation_steps=8)

  return model

In [ ]:
if __name__ == '__main__':
  model = solution_A3()

Epoch 1/20
8/8 [==============================] - 124s 16s/step - loss: 1.9890 - acc: 0.7070 - val_loss: 0.2086 - val_acc: 0.9336
Epoch 2/20
8/8 [==============================] - 49s 7s/step - loss: 0.1517 - acc: 0.9537 - val_loss: 0.0982 - val_acc: 0.9648
Epoch 3/20
8/8 [==============================] - 31s 4s/step - loss: 0.0670 - acc: 0.9868 - val_loss: 0.0702 - val_acc: 0.9844
Epoch 4/20
8/8 [==============================] - 31s 4s/step - loss: 0.0668 - acc: 0.9824 - val_loss: 0.0612 - val_acc: 0.9883
Epoch 5/20
8/8 [==============================] - 22s 3s/step - loss: 0.0486 - acc: 0.9780 - val_loss: 0.0400 - val_acc: 0.9922
Epoch 6/20
8/8 [==============================] - 19s 3s/step - loss: 0.0213 - acc: 0.9971 - val_loss: 0.0738 - val_acc: 0.9766
Epoch 7/20
8/8 [==============================] - 22s 3s/step - loss: 0.1602 - acc: 0.9383 - val_loss: 0.0197 - val_acc: 0.9922
Epoch 8/20
8/8 [==============================] - 18s 2s/step - loss: 0.0102 - acc: 0.9990 - val_loss:

In [ ]:
from keras.preprocessing import image
import numpy as np

img_path = 'images.jpg'  # Ganti dengan path gambar yang diunggah
img = image.load_img(img_path, target_size=(150, 150))  # Sesuaikan dengan ukuran input model Anda
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Sesuaikan normalisasi jika diperlukan

predictions = model.predict(img_array)

if predictions[0][0] > 0.5:
    print("semi-ripe banana")
else:
    print("full-ripe banana")

1/1 [==============================] - 0s 27ms/step
full-ripe banana
